## Load Model From huggingface

In [1]:
from transformers import AutoTokenizer, AutoModel

device = 'cuda:1'
checkpoint = "THUDM/chatglm-6b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True)


/home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.35it/s]


## Insert LoRA to model

In [2]:
import loralib as lora
import lora_utils.insert_lora


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116_nocublaslt.so...


/home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/jiangjin/miniconda3/envs/visgpt did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [3]:
lora_config = {
        'r': 32,
        'lora_alpha':32,
        'lora_dropout':0.1,
        'enable_lora':[True, True, True],
    }

In [4]:
model = lora_utils.insert_lora.get_lora_model(model, lora_config)

trainable_params:22020096 (0.36%), non_trainable_params:6173286400


## Dataset

In [8]:
import dataset.GLM as GLM_Data
from torch.utils.data import DataLoader
device = 'cuda:1'
GLM_Data.device = device

In [9]:
pairs = [{'prompt':'Hello!', 'completion':'Hi! This is ChatGLM.'}]
pairs_encoded = GLM_Data.encode_pairs(pairs, tokenizer)
train_dataset = GLM_Data.GLMDataset(pairs_encoded)
train_dataloader = DataLoader(dataset=train_dataset, collate_fn = GLM_Data.collate_fn, shuffle=True, batch_size=1)

In [10]:
model.half().to(device)
batch = {k: v.to(device) for k, v in next(iter(train_dataloader)).items()}
outputs = model(**batch)

In [11]:
outputs.loss

tensor(3.6367, device='cuda:1', dtype=torch.float16, grad_fn=<ToCopyBackward0>)